In [ ]:
import sys
import time
import traceback
import os
import csv
import pandas as pd
import plotly.graph_objs as go
from dash import Dash, dcc, html
from dash.dependencies import Output, Input
from arduino_iot_cloud import ArduinoCloudClient
from threading import Thread
from datetime import datetime
import cv2

# ---------------------------------------
# ARDUINO CLOUD CREDENTIALS
# ---------------------------------------
DEVICE_ID = "52083cce-bb8f-47bc-a296-3f1de40c0ffa"
SECRET_KEY = "ve9TLiRMv8LeE80zPYJeTobhC"

# ---------------------------------------
# GLOBAL VARIABLES
# ---------------------------------------
cur_data = []
csv_file = "accelerometer.csv"
x, y, z = 0, 0, 0
SNAPSHOT_INTERVAL = 10
_last_snapshot = time.time()
MAX_RUNTIME = 1800

# ---------------------------------------
# SETUP WEBCAM FOR WINDOWS
# ---------------------------------------
# Use DirectShow (Windows-specific)
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    print("Tip: On Windows, check if the webcam is used by another app and allow camera access in Settings.")
    sys.exit(1)

time.sleep(2)
for _ in range(5):
    cap.read()

# ---------------------------------------
# CALLBACKS FOR CLOUD VARIABLES
# ---------------------------------------
def on_accelerometer_x_changed(client, value):
    global x
    x = value

def on_accelerometer_y_changed(client, value):
    global y
    y = value

def on_accelerometer_z_changed(client, value):
    global z
    z = value

# ---------------------------------------
# DATA STREAM THREAD
# ---------------------------------------
def start_data_stream():
    global cur_data, x, y, z, _last_snapshot, cap
    start_time = time.time()

    try:
        client = ArduinoCloudClient(
            device_id=DEVICE_ID,
            username=DEVICE_ID,
            password=SECRET_KEY,
            sync_mode=True
        )

        client.register("py_x", value=None, on_write=on_accelerometer_x_changed)
        client.register("py_y", value=None, on_write=on_accelerometer_y_changed)
        client.register("py_z", value=None, on_write=on_accelerometer_z_changed)
        client.start()

        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            if os.stat(csv_file).st_size == 0:
                writer.writerow(['Timestamp', 'X', 'Y', 'Z'])

        while True:
            if time.time() - start_time > MAX_RUNTIME:
                print("Max runtime reached. Stopping program.")
                break

            if x is not None and y is not None and z is not None:
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
                data_point = [timestamp, x, y, z]
                cur_data.append(data_point)

                if len(cur_data) > 50:
                    cur_data.pop(0)

                with open(csv_file, mode='a', newline='') as file:
                    csv.writer(file).writerow(data_point)

                print(data_point)
                x, y, z = None, None, None

            now = time.time()
            if now - _last_snapshot >= SNAPSHOT_INTERVAL:
                try:
                    time.sleep(0.05)
                    cap.grab()
                    ret, frame = cap.retrieve()

                    if not ret or frame is None or frame.shape[0] == 0:
                        print("Warning: grab() or retrieve() failed — retrying...")
                        cap.release()
                        time.sleep(1)
                        cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
                        time.sleep(1)
                        cap.grab()
                        ret, frame = cap.retrieve()

                    if ret and frame is not None and frame.shape[0] > 0:
                        ts = datetime.now().strftime('%Y%m%dT%H%M%S')
                        img_name = f"seq_{ts}.jpg"
                        cv2.imwrite(img_name, frame)
                        print(f"Snapshot saved: {img_name}")
                    else:
                        print("Error: frame invalid even after retry.")

                except Exception as e:
                    print(f"Exception during snapshot: {e}")

                _last_snapshot = now

            client.update()

    except Exception:
        print("Error in data stream:")
        traceback.print_exc()

# ---------------------------------------
# SMOOTHING FUNCTION
# ---------------------------------------
def smooth_data(df, window_size=5):
    df_smoothed = df.copy()
    for col in ['X', 'Y', 'Z']:
        df_smoothed[col] = df[col].rolling(window=window_size, min_periods=1).mean()
    return df_smoothed

# ---------------------------------------
# DASHBOARD SETUP
# ---------------------------------------
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Live Accelerometer Data (Last 10 Seconds)"),
    dcc.Graph(id='live-graph'),
    dcc.Interval(id='interval-component', interval=1000, n_intervals=0)
])

@app.callback(
    Output('live-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):
    if not cur_data:
        return go.Figure()

    df = pd.DataFrame(cur_data, columns=['Timestamp', 'X', 'Y', 'Z'])
    df = smooth_data(df, window_size=5)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Timestamp'], y=df['X'], mode='lines+markers', name='X'))
    fig.add_trace(go.Scatter(x=df['Timestamp'], y=df['Y'], mode='lines+markers', name='Y'))
    fig.add_trace(go.Scatter(x=df['Timestamp'], y=df['Z'], mode='lines+markers', name='Z'))

    fig.update_layout(
        xaxis_title='Timestamp',
        yaxis_title='Acceleration',
        margin=dict(l=40, r=20, t=40, b=40),
        height=600
    )
    return fig

# ---------------------------------------
# START EVERYTHING
# ---------------------------------------
if __name__ == "__main__":
    data_thread = Thread(target=start_data_stream, daemon=True)
    data_thread.start()
    app.run(debug=False, use_reloader=False, port=8054)
    cap.release()


['2025-05-05 17:10:40.850631', 0, 0, 0]
['2025-05-05 17:10:40.907579', -0.11700000613927841, -0.10200000554323196, 9.838050842285156]
['2025-05-05 17:10:41.912618', -0.10905000567436218, -0.10695000737905502, 9.844950675964355]
['2025-05-05 17:10:42.945431', -0.1189500093460083, -0.08295000344514847, 9.826050758361816]
Snapshot saved: seq_20250505T171045.jpg
['2025-05-05 17:10:46.981223', -0.12600000202655792, -0.10005000233650208, 9.831000328063965]
['2025-05-05 17:10:48.011490', -0.13095000386238098, -0.10500000417232513, 9.838050842285156]
['2025-05-05 17:10:50.046333', -0.12105000764131546, -0.10200000554323196, 9.82395076751709]
['2025-05-05 17:10:51.072544', -0.11700000613927841, -0.08100000023841858, 9.819000244140625]
['2025-05-05 17:10:52.116212', -0.1380000114440918, -0.10500000417232513, 9.832950592041016]
Snapshot saved: seq_20250505T171053.jpg
['2025-05-05 17:10:54.134585', -0.1189500093460083, -0.10005000233650208, 9.82800006866455]
['2025-05-05 17:10:55.166905', -0.12405